In [1]:
import config as cfg
import utils.helper_functions as hf
import utils.inputs as inputs
import utils.outputs as outputs
import utils.model as model

from IPython.display import display, clear_output

import pandas as pd
import numpy as np

import hyperopt
from hyperopt import hp, fmin, tpe

import os

os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1' # disable file validation in the debugger
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #0: All logs (default setting), 1: Filter out INFO logs, up to 3
pd.options.mode.copy_on_write = True # avoid making unnecessary copies of DataFrames or Series


In [2]:
if cfg.use_hyperopt:
    print(cfg.hyperopt_n_iterations)
else:
    num_combinations = hf.get_num_combinations(cfg.param_grid) 

number of combinations: 1


In [3]:
df = pd.read_pickle(cfg.db_path)
df = hf.get_rows_after_date(df, cfg.start_date)
df = hf.fillnavalues(df)

def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

num_tickers = hf.get_num_tickers(get_single_level_df(df, 'Open'))
print(f'number of tickers: {num_tickers}')


number of tickers: 593


In [4]:
def get_df_data(hyperparams):
    df_buy = get_single_level_df(df, hyperparams['buying_time'])
    df_sell = get_single_level_df(df, hyperparams['selling_time'])
    dfs_ohlcv = get_ohlcv_dfs(df)

    if os.path.exists(cfg.transformed_data_path) and cfg.use_saved_transformed_data:
        df_data = pd.read_pickle(cfg.transformed_data_path)
        print(f'using existing {cfg.transformed_data_path}')
    else:
        print(f'need to create {cfg.transformed_data_path}')
        df_data = inputs.get_inputs(df_buy, dfs_ohlcv, hyperparams['buying_time'])
        
        df_data.to_pickle(cfg.transformed_data_path)
        print(f'saved new {cfg.transformed_data_path}')

    df_data = outputs.add_outputs(df_data, df_buy, df_sell, dfs_ohlcv, num_tickers, cfg.output_class_name, cfg.fee, **hyperparams)

    df_data = df_data.dropna()

    return df_data

In [6]:
def slice_df_test(df_data, test_size):
    return df_data.tail(test_size)

def add_predictions(df, model, X_test, **hyperparams):
    print(f'X_test shape: {X_test.shape}')
    
    predicted_n_first_classes = hyperparams['n_first_classes'][0]
    cumulated_probs_target = hyperparams['cumulated_probs_target']

    prediction_y_test_lists = model.predict(X_test)
    prediction_y_test_array = np.array(prediction_y_test_lists)
    df['prediction_probs'] = prediction_y_test_array.tolist()

    df['prediction_cumulated_probs'] = [sum(row[:predicted_n_first_classes+1]) for row in df['prediction_probs']]
    df['prediction_is_buy'] = (df['prediction_cumulated_probs'] > cumulated_probs_target)
    df['prediction_is_buy_is_correct'] = (df['output_is_buy'] == df['prediction_is_buy'])

    return df

def get_class_cumulative_percentages(y_test):
    unique_values, counts = np.unique(y_test, return_counts=True)
    percentages = counts / len(y_test)
    percentages = percentages[np.argsort(unique_values)]
    cumulative_percentages = np.cumsum(percentages)

    print(f'market cumulative % per class: {cumulative_percentages}')

    return cumulative_percentages

def get_market_rate(y_test, **hyperparams):
    accepted_n_first_classes = hyperparams['n_first_classes'][1]

    class_cumulative_percentages = get_class_cumulative_percentages(y_test)
    market_rate = class_cumulative_percentages[accepted_n_first_classes]

    return market_rate

def get_binary_classification(df):
    # tp: true positive, tn: true negative, fp: false positive, fn: false negative  
    tp = ((df['output_is_buy'] == True) & (df['prediction_is_buy'] == True)).sum()
    tn = ((df['output_is_buy'] == False) & (df['prediction_is_buy'] == False)).sum()
    fp = ((df['output_is_buy'] == False) & (df['prediction_is_buy'] == True)).sum()
    fn = ((df['output_is_buy'] == True) & (df['prediction_is_buy'] == False)).sum()

    winning_rate = float(tp / (tp + fp)) if (tp + fp) > 0 else 0

    return {
        'true_positives': tp, 'true_negatives': tn,
        'false_positives': fp, 'false_negatives': fn,
        'winning_rate': winning_rate
    }

def get_profits(df_prediction_is_buy):
    trimmed_average_profit = hf.get_trimmed_average(df_prediction_is_buy['output_profit'], pct_to_trim=0.03, min_num_to_trim=8)
    average_profit = df_prediction_is_buy['output_profit'].mean()
    median_profit = df_prediction_is_buy['output_profit'].median()

    return {
        'trimmed_average_profit': trimmed_average_profit,
        'average_profit': average_profit,
        'median_profit': median_profit
    }

def get_loss_limit_pct(df):
    return df['output_is_loss_limit_reached'].sum() / len(df) if len(df) > 0 else 0

def get_performance_score(trimmed_average_profit, is_buy_count):
    estimated_days = cfg.test_size / num_tickers
    adjusted_profit = trimmed_average_profit ** 8 # to decrease small values, e.g. 0.8^2 = 0.64
    performance_score = adjusted_profit * min(is_buy_count, estimated_days)
    
    return performance_score

def evaluate_model(df_data, model, test_train_data, hyperparams):
    df_test = slice_df_test(df_data, cfg.test_size)
    df_test = add_predictions(df_test, model, test_train_data['X_test'], **hyperparams)
    
    market_rate = get_market_rate(test_train_data['y_test'], **hyperparams)

    binary_classification = get_binary_classification(df_test)
    
    df_prediction_is_buy = df_test[(df_test['prediction_is_buy'] == True)]
    if (not cfg.use_hyperopt and num_combinations == 1):
        print(df_prediction_is_buy.to_markdown())
        df_prediction_is_buy.to_excel(f'./outputs/{hf.get_date()}_classifier_df_prediction_is_buy.xlsx')

    profits = get_profits(df_prediction_is_buy)
    prediction_is_buy_count = len(df_prediction_is_buy['output_profit'])
    loss_limit_reached_pct = get_loss_limit_pct(df_prediction_is_buy)
    performance_score = get_performance_score(profits['trimmed_average_profit'],
                                              prediction_is_buy_count)

    performance_metrics = {
        'performance_score': performance_score,
        **profits,
        'prediction_is_buy_count': prediction_is_buy_count,
        'loss_limit_reached_pct': loss_limit_reached_pct,
        'market_rate': market_rate,
        **binary_classification,
        'winning_rate_vs_market': binary_classification['winning_rate'] - market_rate,
    }

    return performance_metrics

In [7]:
from itertools import product

i = 0
results = []

def objective(hyperparams):
    hyperparams['thresholds'] = [hyperparams['thresholds']]
    hyperparams['rank_pct_thresholds'] = [hyperparams['rank_pct_thresholds']]
    
    df_data = get_df_data(hyperparams)
    test_train_data, model = model.load_model(df_data, hyperparams)
    performance_metrics = evaluate_model(df_data, model, test_train_data, hyperparams)

    result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
    print(result)
    results.append(result)

    performance = result['performance_score']

    return -performance

if cfg.use_hyperopt:
    best = fmin(objective, cfg.search_space, algo=tpe.suggest, max_evals=cfg.hyperopt_n_iterations)
    print(f'best parameters: {best}')
else:
    for params in product(*cfg.param_grid.values()):
        i += 1
        # clear_output(wait=True) # clear printed outputs
        hf.print_combination(i, num_combinations)

        hyperparams = dict(zip(cfg.param_grid.keys(), params))

        df_data = get_df_data(hyperparams)
        test_train_data, model = model.load_model(df_data, hyperparams)
        performance_metrics = evaluate_model(df_data, model, test_train_data, hyperparams)

        result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
        print(result)
        results.append(result)


step: 1/1
need to create ./outputs/classifier_transformed_data.pkl
saved new ./outputs/classifier_transformed_data.pkl
number of elements in y_train: 1605684
number of elements in y_test: 60000
need to create ./outputs/classifier_model.keras


/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
50178/50178 ━━━━━━━━━━━━━━━━━━━━ 31s 606us/step - accuracy: 0.5810 - loss: 1.2822 - val_accuracy: 0.5640 - val_loss: 0.6785
Epoch 2/2
50178/50178 ━━━━━━━━━━━━━━━━━━━━ 31s 610us/step - accuracy: 0.5962 - loss: 1.2355 - val_accuracy: 0.6395 - val_loss: 0.6568
X_test shape: (60000, 52)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
market cumulative % per class: [0.06546667 1.        ]
|                                                    |   input_var_past_90d_future_0d |   input_var_past_60d_future_0d |   input_var_past_30d_future_0d |   input_var_past_10d_future_0d |   input_var_past_5d_future_0d |   input_var_past_2d_future_0d |   input_var_past_1d_future_0d |   input_var_past_close_1d |   input_var_past_high_1d |   input_var_past_low_1d |   input_volume_var_90-1d |   input_volume_var_60-1d |   input_volume_var_30-1d |   input_volume_var_10-1d |   input_volume_var_2-1d |   input_volume_var_3-1d |   input_min_var_past_90d |   input_min_var_past_30d |   input_min_var_past_10d |  

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='performance_score', ascending=False)
df_results.head(1000)

,performance_score,trimmed_average_profit,average_profit,median_profit,prediction_is_buy_count,loss_limit_reached_pct,market_rate,true_positives,true_negatives,false_positives,false_negatives,winning_rate,winning_rate_vs_market,buying_time,selling_time,target_future_days,loss_limit,sell_at_target,size_layer_1,size_layer_2,size_layer_3,dropout_rate,balance_data,batch_size,n_first_classes,cumulated_probs_target,thresholds,rank_pct_thresholds,epochs
0,104.958905,1.015236,1.043741,0.99302,93,0.473118,0.065467,29,56008,64,3899,0.311828,0.246361,Open,Open,1,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.0184],[0.45],2


In [9]:
df_results.to_excel(f'./outputs/{hf.get_date()}_classifier_results.xlsx')